<a href="https://colab.research.google.com/github/Abinaya-J/Self_Driving_Car/blob/main/P3_Behavioral_Cloning_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Load data**

In [ ]:
from google.colab import files
files.upload()

Saving drive.py to drive.py


{'drive.py': b'import argparse\nimport base64\nfrom datetime import datetime\nimport os\nimport shutil\n\nimport numpy as np\nimport socketio\nimport eventlet\nimport eventlet.wsgi\nfrom PIL import Image\nfrom flask import Flask\nfrom io import BytesIO\n\nfrom keras.models import load_model\nimport h5py\nfrom keras import __version__ as keras_version\n\nsio = socketio.Server()\napp = Flask(__name__)\nmodel = None\nprev_image_array = None\n\n\nclass SimplePIController:\n    def __init__(self, Kp, Ki):\n        self.Kp = Kp\n        self.Ki = Ki\n        self.set_point = 0.\n        self.error = 0.\n        self.integral = 0.\n\n    def set_desired(self, desired):\n        self.set_point = desired\n\n    def update(self, measurement):\n        # proportional error\n        self.error = self.set_point - measurement\n\n        # integral error\n        self.integral += self.error\n\n        return self.Kp * self.error + self.Ki * self.integral\n\n\ncontroller = SimplePIController(0.1, 0.00

In [ ]:
!python drive.py

Traceback (most recent call last):
  File "drive.py", line 8, in <module>
    import socketio
ModuleNotFoundError: No module named 'socketio'


In [ ]:
!unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_298.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_28_400.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_400.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_28_502.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_502.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_28_604.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_604.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_28_705.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_705.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_28_806.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_806.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_28_906.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_28_906.jpg  
  inflating: data/IMG/right_2016_12_01_13_42_29_007.jpg  
  inflating: __MACOSX/data/IMG/._right_2016_12_01_13_42_29_0

### Create train and test data

In [ ]:
import cv2
import csv
import pandas as pd
import numpy as np

In [ ]:
alldata = pd.read_csv('data/driving_log.csv')
display(alldata[0:5])

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [ ]:
lines=[]
with open('data/driving_log.csv') as csvfile:
  reader = csv.reader(csvfile)
  for line in reader:
    lines.append(line)

In [ ]:
images=[]
steering_m=[]
for line in lines[1:]:
  for i in range(3):
    path=line[i]
    filename= path.split('/')[-1]
    newpath = 'data/IMG/' + filename
    image = cv2.imread(newpath)
    images.append(image)
    sm = float(line[3])
    steering_m.append(sm)

In [ ]:
X_train = np.array(images)
Y_train = np.array(steering_m)

In [ ]:
print(X_train[0].shape)

(160, 320, 3)


In [ ]:
print(X_train[0])

[[[208 163 130]
  [208 163 130]
  [208 163 130]
  ...
  [210 163 131]
  [210 163 131]
  [210 163 131]]

 [[208 163 130]
  [208 163 130]
  [208 163 130]
  ...
  [210 163 131]
  [210 163 131]
  [210 163 131]]

 [[209 164 131]
  [209 164 131]
  [209 164 131]
  ...
  [211 164 132]
  [211 164 132]
  [211 164 132]]

 ...

 [[ 83  99 112]
  [ 88 104 117]
  [ 90 106 119]
  ...
  [ 99 117 134]
  [108 123 142]
  [111 129 146]]

 [[ 92 109 122]
  [ 88 105 118]
  [ 82  98 114]
  ...
  [104 124 142]
  [ 98 115 136]
  [ 91 111 129]]

 [[ 87 104 117]
  [ 79  96 109]
  [ 79  95 111]
  ...
  [115 134 155]
  [106 125 146]
  [ 99 118 139]]]


### Train and validate model

In [ ]:
for i in range(len(X_train)):
  X_train[i] = X_train[i]/255 -0.5

In [ ]:

import keras 
import pandas as pd
import tensorflow as tf
import cv2

#Processing input
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input

# Modelling
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, UpSampling2D, Input
from keras.initializers import glorot_uniform
from keras.optimizers import SGD , Adam
from keras.callbacks import LearningRateScheduler
from keras import regularizers
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [ ]:
def data():
    batch_size=32
    train_datagen = ImageDataGenerator(shear_range=0.2,
                                       rotation_range=20.,
                                       width_shift_range=0.3,
                                       height_shift_range=0.3,
                                       zoom_range=0.2)
    
    train_generator = train_datagen.flow(X_train,Y_train, batch_size=batch_size, shuffle=True)

    return train_generator

train_generator=data()

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', padding = 'same', input_shape=(160,320, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3,3), padding = 'same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(128,activation= 'relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64,activation= 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(43,activation= 'softmax')])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer = Adam(learning_rate=0.0006), metrics=["accuracy"])